# BG/NBD model

Comparison with lifetimes

**Reference**: Fader, P. S., Hardie, B. G., & Lee, K. L. (2005). “Counting your customers” the easy way: An alternative to the Pareto/NBD model. Marketing science, 24(2), 275-284.

http://www.brucehardie.com/papers/bgnbd_2004-04-20.pdf

In [7]:
import arviz as az
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from pymc_marketing import clv

In [3]:
from lifetimes.datasets import load_cdnow_summary
from lifetimes import BetaGeoFitter

In [4]:
df = load_cdnow_summary(index_col=[0])
df.head()

,frequency,recency,T
ID,,,
1,2,30.43,38.86
2,1,1.71,38.86
3,0,0.00,38.86
4,0,0.00,38.86
5,0,0.00,38.86


In [12]:
data = pd.DataFrame({
    'customer_id': np.arange(len(x)),
    'frequency': df["frequency"].values,
    'recency': df["recency"].values,
    'T': df['T'].values,
})

model_config = {
    "a_prior": {"dist": "HalfNormal", "kwargs": {}},
    "b_prior": {"dist": "HalfStudentT", "kwargs": {"nu": 4}},
    "alpha_prior": {"dist": "HalfCauchy", "kwargs": {"beta": 2}},
    "r_prior": {"dist": "Gamma", "kwargs": {"alpha": 1, "beta": 1}},
}

In [13]:
model = clv.BetaGeoModel(data = data, model_config = model_config)
model.build_model()
model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [a, b, alpha, r]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 10 seconds.
/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/site-packages/pymc_experimental/model_builder.py:297: UserWarning: The effect of Potentials on other parameters is ignored during prior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  idata.extend(pm.sample_prior_predictive())
Sampling: [a, alpha, b, r]
/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/site-packages/pymc_experimental/model_builder.py:298: UserWarning: The effect of Potentials on other parameters is ignored during posterior predictive sampling. This is likely to lead to invalid or biased predictive samples.
  idata.extend(pm.sample_posterior_predictive(idata))
/Users/michalraczycki/Documents/pymc-marketing/.conda/envs/marketing_env/lib/python3.11/site-packages/arviz/data/inference_data.py:1459: UserWarning

Inference data with groups:
	> posterior
	> sample_stats
	> prior
	> fit_data

In [ ]:
model.fit

In [7]:
model.fit_summary()

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
a,0.980,0.295,0.515,1.474,0.008,0.006,1618.0,1426.0,1.0
b,3.228,1.249,1.388,5.273,0.035,0.027,1632.0,1389.0,1.0
alpha,4.496,0.379,3.847,5.254,0.009,0.006,1852.0,1981.0,1.0
r,0.245,0.012,0.221,0.268,0.000,0.000,1838.0,1935.0,1.0


### Using MAP fit

`CLVModel`s, which includes `BetaGeoModel`, can provide the maximum a posteriori estimates using a numerical optimzer from `scipy.optimize`.

In [8]:
model.fit(fit_method="map") # results from previous fit are overriden

/Users/larryshamalama/Documents/pymc-dev/pymc-marketing/pymc_marketing/clv/models/basic.py:92: UserWarning: Overriding pre-existing fit_result
  warnings.warn("Overriding pre-existing fit_result")


Inference data with groups:
	> posterior

In [9]:
model.fit_summary()

a        0.793
b        2.426
alpha    4.414
r        0.243
Name: value, dtype: float64

### Comparing with the `lifetimes` package

In [10]:
bgf = BetaGeoFitter()
bgf.fit(frequency=x, recency=t_x, T=T)
bgf.summary

,coef,se(coef),lower 95% bound,upper 95% bound
r,0.242593,0.012557,0.217981,0.267205
alpha,4.413532,0.378221,3.672218,5.154846
a,0.792886,0.185719,0.428877,1.156895
b,2.425752,0.705345,1.043276,3.808229
